In [1]:
import sys, os
import pandas as pd
import pathSetting
sys.path.append("../../..")

from Clust.clust.ML.common.common import p1_integratedDataSaving as p1
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import clean as ml_clean
from Clust.clust.ML.tool import meta as ml_meta

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

import torch

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

#import main_regression as mr
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


In [2]:
# Set regression mode i.e., 'regression','forecast' 
mode_selection = "forecast"

# Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 
model_method = 'LSTM_rg'

# get integrated data name
bucket_name = 'integration'

# scaler path
scalerPath = './scaler/'

# 2. Training 

## 2-1. Data selection

In [3]:
get_ms_list = db_client.measurement_list(bucket_name)
get_ms_list

/home/leezy/anaconda3/lib/python3.8/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    import "influxdata/influxdb/schema" schema.measurements(bucket: "integration") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


['forecasting_Hs2SwineFarmWithWeatherTime_test_Clean',
 'forecasting_Hs2SwineFarmWithWeatherTime_test_NoClean',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_Clean',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_NoClean',
 'forecasting_gunwiStrawberryWeather_test_Clean',
 'forecasting_gunwiStrawberryWeather_test_NoClean',
 'forecasting_gunwiStrawberryWeather_train_Clean',
 'forecasting_gunwiStrawberryWeather_train_NoClean',
 'forecasting_strawberryOpen_test_Clean',
 'forecasting_strawberryOpen_test_NoClean',
 'forecasting_strawberryOpen_train_Clean',
 'forecasting_strawberryOpen_train_NoClean',
 'regression_energy_testX_Clean',
 'regression_energy_testX_NoClean',
 'regression_energy_testy_Clean',
 'regression_energy_testy_NoClean',
 'regression_energy_trainX_Clean',
 'regression_energy_trainX_NoClean',
 'regression_energy_trainy_Clean',
 'regression_energy_trainy_NoClean']

In [4]:
collection_list = mongo_client.get_collection_list(bucket_name)
collection_list

['forecasting_Hs2SwineFarmWithWeatherTime',
 'regression_energy',
 'forecasting_strawberryOpen',
 'forecasting_gunwiStrawberryWeather']

In [7]:
# for regression data
if mode_selection == 'regression':
    # dataX
    data_name_X = 'regression_energy_trainX_NoClean'
    dataX = db_client.get_data(bucket_name, data_name_X)

    # datay
    data_name_y = 'regression_energy_trainy_NoClean'
    datay = db_client.get_data(bucket_name, data_name_y)

    dataset_name = collection_list[1]
    data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'data_name':data_name_X})[0]
    cleanMode = data_meta["clean_param"]
    integration_freq_sec = data_meta["integration_param"]["integration_frequency"]

# for forecast data test
elif mode_selection == 'forecast':

    data_name_X = 'forecasting_Hs2SwineFarmWithWeatherTime_train_NoClean'
    dataX = db_client.get_data(bucket_name, data_name_X)
    
    dataset_name = collection_list[0]
    data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'data_name':data_name_X})[0]
    cleanMode = data_meta["clean_param"]
    integration_freq_sec = data_meta["integration_param"]["integration_frequency"]

In [8]:
print(type(dataX), dataX.shape)
print(dataX)

<class 'pandas.core.frame.DataFrame'> (10645, 20)
                                CO2ppm    H2Sppm   Humidity      NH3ppm  \
time                                                                      
2021-02-01 00:00:00+00:00  2395.666667  0.666667  65.200000   14.000000   
2021-02-01 00:05:00+00:00  2378.500000  1.000000  65.200000   14.000000   
2021-02-01 00:10:00+00:00  2385.000000  1.000000  65.100000   14.000000   
2021-02-01 00:15:00+00:00  2385.500000  1.000000  65.100000   14.000000   
2021-02-01 00:20:00+00:00  2358.333333  1.000000  65.000000   14.000000   
...                                ...       ...        ...         ...   
2021-03-09 22:40:00+00:00  3355.333333  3.000000  61.333333  129.666667   
2021-03-09 22:45:00+00:00  3384.000000  3.000000  61.400000  130.500000   
2021-03-09 22:50:00+00:00  3453.000000  2.666667  61.533333  133.333333   
2021-03-09 22:55:00+00:00  3522.500000  3.000000  61.900000  134.000000   
2021-03-09 23:00:00+00:00  3541.000000  4.000000  

In [ ]:
# print(type(datay), datay.shape)
# print(datay)

## 2-2. Training Data Preparation

In [9]:
featureListX= list(dataX.columns)
featureListX

['CO2ppm',
 'H2Sppm',
 'Humidity',
 'NH3ppm',
 'Temperature',
 'cos_day_of_week',
 'cos_hour',
 'cos_month',
 'cos_week_of_year',
 'out_humid',
 'out_pressure',
 'out_rainfall',
 'out_sunshine',
 'out_temp',
 'out_wind_direction',
 'out_wind_speed',
 'sin_day_of_week',
 'sin_hour',
 'sin_month',
 'sin_week_of_year']

In [10]:
# for regression
if mode_selection == 'regression':
    # 2 Training Data Preparation
    # 2-1
    featureListX= list(dataX.columns)
    featureListy= list(datay.columns)

    # 2-2
#     cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함
    cleanTrainDataParam = cleanMode
    
    # 2-2-1 cleanTrainDataParam == Clean 일 경우
    NaNProcessingParam ={
        "feature_cycle":'Day',
        "feature_cycle_times":1,
        "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
    }
    # 2-3
    scalerParam='scale'
    scaleMethod='minmax'

    # 2-4
    splitRatio = 0.8

    # 2-5
    scalerRootPath_X = os.path.join(scalerPath, data_name_X, cleanTrainDataParam)
    scalerRootPath_y = os.path.join(scalerPath, data_name_y, cleanTrainDataParam)
    train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
    train_y, val_y, y_scalerFilePath = ml_data.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, scalerParam, scaleMethod)

# for forecast
elif mode_selection == 'forecast':
    # 2-1
    featureListX = ['Temperature', 'out_temp','sin_hour']
    featureListy = 'Temperature'

    # 2-2
    cleanTrainDataParam = cleanMode

    # 2-2-1 cleanMode == Clean 일 때만 활성화
    NaNProcessingParam ={
        "feature_cycle":'Day',
        "feature_cycle_times":1,
        "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
    }

    # 2-3
    scalerParam='scale'
    scaleMethod ='robust'

    # 2-4
    splitRatio = 0.8

    # 2-5
    scalerRootPath_X = os.path.join(scalerPath, data_name_X, cleanMode)
    train_x, val_x, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod)
    train_x, val_x = ml_clean.delete_low_quality_train_val_data(train_x, val_x, cleanMode, NaNProcessingParam)

New json file is created from data.json file
['Temperature', 'out_temp', 'sin_hour']
Make New scaler File
------ {'feature_cycle': 'Day', 'feature_cycle_times': 1, 'NanInfoForCleanData': {'type': 'num', 'ConsecutiveNanLimit': 3, 'totalNaNLimit': 30000}}


In [11]:
print(type(train_x), train_x.shape)
print(train_x)

<class 'pandas.core.frame.DataFrame'> (8516, 3)
                           Temperature  out_temp      sin_hour
time                                                          
2021-02-01 00:00:00+00:00    -0.639485 -0.367914 -8.659561e-17
2021-02-01 00:05:00+00:00    -0.639485 -0.365775  1.525106e-02
2021-02-01 00:10:00+00:00    -0.639485 -0.363636  3.050212e-02
2021-02-01 00:15:00+00:00    -0.639485 -0.361497  4.575318e-02
2021-02-01 00:20:00+00:00    -0.639485 -0.359358  6.100423e-02
...                                ...       ...           ...
2021-03-02 13:15:00+00:00     0.120172  1.014973 -2.256479e-01
2021-03-02 13:20:00+00:00     0.107296  0.979679 -2.398596e-01
2021-03-02 13:25:00+00:00     0.107296  0.944385 -2.540713e-01
2021-03-02 13:30:00+00:00     0.120172  0.909091 -2.682831e-01
2021-03-02 13:35:00+00:00     0.120172  0.873797 -2.824948e-01

[8516 rows x 3 columns]


In [12]:
print(type(val_x), val_x.shape)
print(val_x)
# print(type(train_y), train_y.shape)
# print(train_y)

<class 'pandas.core.frame.DataFrame'> (2129, 3)
                           Temperature  out_temp  sin_hour
time                                                      
2021-03-02 13:40:00+00:00     0.128755  0.838503 -0.296706
2021-03-02 13:45:00+00:00     0.120172  0.803209 -0.310918
2021-03-02 13:50:00+00:00     0.120172  0.767914 -0.325130
2021-03-02 13:55:00+00:00     0.120172  0.732620 -0.339342
2021-03-02 14:00:00+00:00     0.128755  0.697326 -0.353553
...                                ...       ...       ...
2021-03-09 22:40:00+00:00     0.064378  0.145455 -0.239860
2021-03-09 22:45:00+00:00     0.087983  0.135829 -0.225648
2021-03-09 22:50:00+00:00     0.107296  0.126203 -0.211436
2021-03-09 22:55:00+00:00     0.139485  0.116578 -0.197224
2021-03-09 23:00:00+00:00     0.158798  0.106952 -0.183013

[2129 rows x 3 columns]


In [13]:
def transDFtoNP_yk(dfX, dfy, windowNum = 0, dim = None):
    """
    Make NumpyArray by input DataFrame.
    if windowNum = 0 ----> slice X by day
    if windowNum = N ----> slice X by windowNum
    
    Example:
        >>> Retunrn 
        ... X.shape (sampleNum, featureNum, sequenceNum )
        ... y.shape (sampleNum, )

    Args:
        dfX (DataFrame): dfX
        dfy (DataFrame): dfy
        windowNum (Interger): windowNum

    Returns:
        numpy array:  X, y
    
    """
    import datetime as dt
    import numpy as np

    if dim == 2:
        X = dfX.to_numpy()
        y = np.array(dfy.squeeze().tolist())
    else:
        X =[]
        y= []

        if windowNum ==0:
            dateList = dfX.index.map(lambda t: t.date()).unique()
            for startDate in dateList:
                endDate  = dt.datetime.combine(startDate, dt.time(23, 59, 59, 59))
                dfX_partial = dfX[startDate:endDate]
                dfy_partial = dfy[startDate:endDate]
                X_partial = dfX_partial.values
                y_partial = dfy_partial.values[0][0]
                X.append (X_partial)
                y.append (y_partial)
        else:
            import math
            roundNum = math.ceil(len(dfX)/windowNum)
            for i in range(roundNum): #This ensures all rows are captured
                dfX_partial = dfX[i*windowNum:(i+1)*windowNum]
                dfy_partial = dfy[i:(i+1)]
                X_partial = dfX_partial.values
                y_partial = dfy_partial.values[0][0]
                X.append (X_partial)
                y.append (y_partial)

        X = np.array(X)
        y = np.array(y)
    
    return X, y

## 2-3. Training Data Transformation

In [14]:
# for regression
if mode_selection == 'regression':
    transformParameter = {}

    # from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf

    trainX, trainy = transDFtoNP_yk(train_x, train_y, windowNum=144)
    valX, valy = transDFtoNP_yk(val_x, val_y, windowNum=144)

# for forecast
elif mode_selection == 'forecast':
    transformParameter = {
    'future_step': 2,
    'past_step': 24,
    'feature_col': featureListX,
    'target_col': featureListy,
    'clean_param': cleanTrainDataParam
    }
    from Clust.clust.transformation.purpose.machineLearning import LSTMData

    LSTMD = LSTMData()
    trainX, trainy = LSTMD.transform_Xy_arr(train_x, transformParameter, transformParameter['clean_param'])
    valX, valy = LSTMD.transform_Xy_arr(val_x, transformParameter, transformParameter['clean_param'])

1. Original Data Lenagh: 8514
2. Removed Data Length: 0
3. Clean Data Leangth: 8491
1. Original Data Lenagh: 2127
2. Removed Data Length: 0
3. Clean Data Leangth: 2104


In [15]:
print(type(trainX), trainX.shape)
print(trainX)

<class 'numpy.ndarray'> (8491, 24, 3)
[[[-6.39484979e-01 -3.67914439e-01 -8.65956109e-17]
  [-6.39484979e-01 -3.65775401e-01  1.52510587e-02]
  [-6.39484979e-01 -3.63636364e-01  3.05021173e-02]
  ...
  [-3.49785408e-01 -3.42245989e-01  3.10918223e-01]
  [-3.56223176e-01 -3.42245989e-01  3.25129948e-01]
  [-3.56223176e-01 -3.42245989e-01  3.39341672e-01]]

 [[-6.39484979e-01 -3.65775401e-01  1.52510587e-02]
  [-6.39484979e-01 -3.63636364e-01  3.05021173e-02]
  [-6.39484979e-01 -3.61497326e-01  4.57531760e-02]
  ...
  [-3.56223176e-01 -3.42245989e-01  3.25129948e-01]
  [-3.56223176e-01 -3.42245989e-01  3.39341672e-01]
  [-3.56223176e-01 -3.42245989e-01  3.53553397e-01]]

 [[-6.39484979e-01 -3.63636364e-01  3.05021173e-02]
  [-6.39484979e-01 -3.61497326e-01  4.57531760e-02]
  [-6.39484979e-01 -3.59358289e-01  6.10042347e-02]
  ...
  [-3.56223176e-01 -3.42245989e-01  3.39341672e-01]
  [-3.56223176e-01 -3.42245989e-01  3.53553397e-01]
  [-3.56223176e-01 -3.42245989e-01  3.65757280e-01]]

 .

## 2-4 Set Model Parameters & Train Parameters

In [16]:
seq_len, input_size = trainX.shape[1], trainX.shape[2]
print(input_size, seq_len)

# RNN models (RNN, LSTM, GRU) parameters
if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
    modelParameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': 64,
        'num_layers': 2,
        'output_dim': 1, 
        'dropout': 0.1, 
        'bidirectional': True
    }
# CNN_1D model parameters
elif model_method == 'CNN_1D_rg':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'output_channels': 64,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0, 
    'dropout': 0.1
    }
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_rg':
    modelParameter = {
    'input_size': input_size,
    'num_layers': 2,
    'lstm_dropout': 0.4,
    'fc_dropout': 0.1
    }
# FC model parameters
elif model_method == 'FC_rg':
    modelParameter = {
    'input_size': input_size,
    'dropout': 0.1,
    'bias': True
    }

trainParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 10, 
    'batch_size': 16
}

3 24


In [17]:
modelTags =["model_tag_example"]
trainDataType = "timeseries"
modelPurpose = "regression"

# # 2
trainDataInfo = data_meta

# 3. 모델을 저장할 파일 패스를 생성한다.

model_name = dataset_name + '_' + model_method + '_' + cleanMode
print(model_name)

from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
trainDataPathList = [model_name, data_name_X, trainParameter_encode]
modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

forecasting_Hs2SwineFarmWithWeatherTime_LSTM_rg_NoClean
./Models/LSTM_rg/forecasting_Hs2SwineFarmWithWeatherTime_LSTM_rg_NoClean/forecasting_Hs2SwineFarmWithWeatherTime_train_NoClean/d531eec11664669cff1f6a3ad9639012/model.pkl


## 2-5 Training 

In [18]:
from Clust.clust.ML.regression_YK.train import RegressionTrain as RML

rml = RML()
rml.set_param(trainParameter)
rml.set_model(model_method, modelParameter)
rml.set_data(trainX, trainy, valX, valy)
rml.train()
rml.save_best_model(modelFilePath)

Start training model
[1/10] Training loss: 0.0990	 Validation loss: 0.0033
[2/10] Training loss: 0.0056	 Validation loss: 0.0022
[3/10] Training loss: 0.0042	 Validation loss: 0.0017
[4/10] Training loss: 0.0034	 Validation loss: 0.0014
[5/10] Training loss: 0.0028	 Validation loss: 0.0012
[6/10] Training loss: 0.0025	 Validation loss: 0.0011
[7/10] Training loss: 0.0024	 Validation loss: 0.0011
[8/10] Training loss: 0.0023	 Validation loss: 0.0010
[9/10] Training loss: 0.0022	 Validation loss: 0.0009
[10/10] Training loss: 0.0021	 Validation loss: 0.0010

Training complete in 1m 40s


## 2-6 Save MetaData 

In [19]:
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
# trainParameter_encode =  encode_hash_style(str(trainParameter))
if mode_selection == 'regression':
    modelInfoMeta ={
        "trainDataInfo":trainDataInfo,
        "modelName":model_name,
        "featureList":featureListX,
        "target": featureListy,
        "trainDataType":trainDataType,
        "modelPurpose":modelPurpose,
        "model_method":model_method,
        "modelTags":modelTags,
        "cleanTrainDataParam":cleanTrainDataParam,
        "NaNProcessingParam":NaNProcessingParam,
        "trainDataName":[data_name_X, data_name_y],
        "trainParameter": trainParameter,
        "modelParameter": modelParameter,
        "transformParameter":transformParameter,
        "NaNProcessingParam":NaNProcessingParam,
        "scalerParam":scalerParam,
        "files":{
            "modelFile":{
                "fileName":"model.pth",
                "filePath":modelFilePath
            },
            "XScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":X_scalerFilePath       
            },
            "yScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":y_scalerFilePath       
            }
        }
    }

elif mode_selection == 'forecast':
    modelInfoMeta ={
        "trainDataInfo":trainDataInfo,
        "modelName":model_name,
        "featureList":featureListX,
        "target": featureListy,
        "trainDataType":trainDataType,
        "modelPurpose":modelPurpose,
        "model_method":model_method,
        "modelTags":modelTags,
        "cleanTrainDataParam":cleanTrainDataParam,
        "NaNProcessingParam":NaNProcessingParam,
        "trainDataName":[data_name_X],
        "trainParameter": trainParameter,
        "modelParameter": modelParameter,
        "transformParameter":transformParameter,
        "NaNProcessingParam":NaNProcessingParam,
        "scalerParam":scalerParam,
        "files":{
            "modelFile":{
                "fileName":"model.pth",
                "filePath":modelFilePath
            },
            "XScalerFile":{
                "fileName":"scaler.pkl",
                "filePath":X_scalerFilePath       
            }
        }
    }

modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)


========== Data Save Success ==========
{'trainDataInfo': {'data_name': 'forecasting_Hs2SwineFarmWithWeatherTime_train_NoClean', 'ingestion_param': {'start_time': '2021-02-01 00:00:00', 'end_time': '2021-03-10 00:00:00', 'ms_list_info': [['farm_swine_air', 'HS2'], ['weather_outdoor_keti_clean', 'sangju'], ['life_additional_Info', 'trigonometicInfoByHours']]}, 'integration_param': {'param': {}, 'method': 'meta', 'integration_duration': 'common', 'integration_frequency': 300}, 'clean_param': 'NoClean', 'process_param': {'refine_param': {'removeDuplication': {'flag': False}, 'staticFrequency': {'flag': False, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': False}, 'unCertainErrorToNaN': {'flag': False, 'param': {}}}, 'imputation_param': {'flag': False, 'imputation_method': [], 'totalNonNanRatio': 80}}}, 'modelName': 'forecasting_Hs2SwineFarmWithWeatherTime_LSTM_rg_NoClean', 'featureList': ['Temperature', 'out_temp', 'sin_hour'], 'target': 'Temperature', 'trainDataType